In [1]:
import json

client_info_path = ".\\Credentials\\api_credential_tenant.json"
client_info = json.load(open(client_info_path))
client_id = client_info["client_id"]
client_secret = client_info["client_secret"]

In [2]:
import TenantAPI

tenant = TenantAPI.Tenant(client_id, client_secret)

In [ ]:
# Alerts List
endpoint = "/common/v1/alerts"
alerts_by_severity = {"high": [], "medium": [], "low": []}
base_endpoint = endpoint
alert_keys = ("id", "allowedActions", "category", "description", "product", "raisedAt", "severity", "tenant", "type")

try:
    while True:
        response = tenant.get(base_endpoint)

        if response.get("status") == "success":
            data = response.get("data", {})
            print("Request was successful!")
            alerts = data.get("items", [])
            next_page = data.get("pages", {}).get("nextKey")

            for alert in alerts:
                alert_dict = {key: alert.get(key) for key in alert_keys}
                alerts_by_severity.get(alert.get("severity"), []).append(alert_dict)

            if next_page:
                base_endpoint = f"{endpoint}?pageFromKey={next_page}"
            else:
                break
        else:
            print("Error Message:", response.get("error"), "\nError Details:", response.get("details", "No further details available."))
            break
finally:
    tenant.close()

combined_alert_list = alerts_by_severity["high"] + alerts_by_severity["medium"] + alerts_by_severity["low"]

list_of_high_alerts = alerts_by_severity["high"]
list_of_medium_alerts = alerts_by_severity["medium"]
list_of_low_alerts = alerts_by_severity["low"]

In [ ]:
# Print Alerts

print("All alerts:", len(combined_alert_list))

print("\nHigh Alerts:", len(list_of_high_alerts))
for item in list_of_high_alerts:
    print(item)

print("\nMedium Alerts:", len(list_of_medium_alerts))
for item in list_of_medium_alerts:
    print(item)

print("\nLow Alerts:", len(list_of_low_alerts))
for item in list_of_low_alerts:
    print(item)

In [ ]:
# List of Endpoints

endpoint = "/endpoint/v1/endpoints"
list_of_endpoints = []

with TenantAPI.Tenant(client_id, client_secret) as tenant:
    while True:
        response = tenant.get(endpoint)

        if response.get("status") == "success":
            response_data = response.get("data", {})
            print("Request was successful!")
            list_of_endpoints.extend(response_data.get("items", []))

            next_page = response_data.get("pages", {}).get("nextKey")
            if not next_page:
                break
            endpoint = f"/endpoint/v1/endpoints?pageFromKey={next_page}"
        else:
            print("Error Message:", response.get("message"))
            print("Error Details:", response.get("details", "No further details available."))
            break


for item in list_of_endpoints:
    print(item)

In [19]:
tenant = TenantAPI.Tenant(client_id, client_secret)

In [ ]:
# Get alerts matching criteria in query parameters.

endpoint = "/common/v1/alerts/"

# Filter by product and severity
params = {
    "product": "firewall",
    # "severity": "high",
}

response = tenant.get(endpoint, params=params)

if response.get("status") == "success":
    items = response.get("data", {}).get("items", [])
    for item in items:
        print(item)
else:
    print("Error Message:", response.get("error", "Unknown error"))
    print("Error Details:", response.get("details", "No further details available."))

In [ ]:
# Take an action on a specific alert.

alert_id = 'f632d4f4-740b-427f-9f8a-c64e2ab2b62d'


endpoint = f"/common/v1/alerts/{alert_id}/actions"

body = {
    "action": "acknowledge",
    "message": "Central API Test",
}

response = tenant.post(endpoint, json=body)

if response.get("status") == "success":
    print(json.dumps(response.get("data"), indent=4))
else:
    print("Error Message:", response.get("error", "Unknown error"))
    print("Error Details:", response.get("details", "No further details available."))

In [ ]:
# Get static packages

endpoint = f"/endpoint/v1/software/packages/static"

# Sorted by Release Date
params = {
    "sort": "releasedAt:desc",
}

# params = {"sort": "expiresAt:asc"}
response = tenant.get(endpoint, params=params)

if response.get("status") == "success":
    for item in response.get("data").get("items", []):
        print(item)
else:
    print("Error Message:", response.get("error", "Unknown error"))
    print("Error Details:", response.get("details", "No further details available."))

In [10]:
tenant.close()

In [ ]:
headers = {
    # "X-Tenant-ID": 'tenant_id',
    "Authorization": f"Bearer {'access_token'}",
    "Accept": "application/json",
    "Content-Type": "application/json",
}

headers.pop("X-Tenant-ID", None)
headers.pop("X-Partner-ID", None)
headers.pop("X-Organization-ID", None)


headers |= {"X-Partner-ID": "partner_id"}


print(headers)

In [ ]:
headers